In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from IPython.display import display

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Set logging level to WARNING

In [3]:
# File path
file_path = '../input/power_data.csv'

In [ ]:
# Ensure export directory exists
os.makedirs(output_path, exist_ok=True)

In [4]:
# Load the input into memory using Pandas (for large dataset it's better to use Dask)
df = pd.read_csv(file_path)

In [5]:
# Specialized function for rolling median
def fill_missing_with_rolling_median(data, date_column, target_column, rolling_window_days):
    """
    Fills missing values in a target column using the median of a rolling window.

    Args:
        data (pd.DataFrame): The DataFrame containing the input.
        date_column (str): The column name for the datetime information.
        target_column (str): The column with missing values to fill.
        rolling_window_days (int): Number of days for the rolling window.

    Returns:
        pd.Series: A series with filled missing values.
    """
    # Convert date_column to datetime if not already
    if not np.issubdtype(data[date_column].dtype, np.datetime64):
        data[date_column] = pd.to_datetime(data[date_column])

    filled_column = data[target_column].copy()

    # Use rolling window with datetime-based indexing for efficiency
    rolling_window = f"{rolling_window_days}D"
    filled_column = filled_column.fillna(
        data.set_index(date_column)[target_column].rolling(rolling_window, min_periods=1).median()
    )

    return filled_column

In [6]:
# Handle missing values
def handle_missing_values(data, threshold=5, fill_method='median', rolling_window_days=90, date_column='date'):
    """
    Handles missing values in a DataFrame based on a specified threshold and filling method.

    Args:
    input (pd.DataFrame): The DataFrame to process.
    threshold (float): Percentage threshold to decide whether to fill or drop columns.
    fill_method (str): Method to fill missing values ('mean', 'median', or 'rolling_median').
    rolling_window_days (int): Rolling window size in days (used only if fill_method='rolling_median').
    date_column (str): The column name containing datetime information.

    Returns:
    tuple: (Processed DataFrame, DataFrame of results for tabulation)
    """
    total_rows = len(data)
    results = []  # Store results for tabulation

    for column in data.columns:
        missing_count = data[column].isna().sum()
        missing_percentage = (missing_count / total_rows) * 100

        if missing_percentage == 0:
            results.append({
                "Column": column,
                "Missing Count": missing_count,
                "Missing Percentage": missing_percentage,
                "Action": "No Missing Values"
            })
            continue  # No missing values to handle

        action = ""

        if missing_percentage < threshold:
            if fill_method == 'mean':
                fill_value = data[column].mean()
                data[column] = data[column].fillna(fill_value)
                action = f"Filled with mean ({fill_value:.2f})"
            elif fill_method == 'median':
                fill_value = data[column].median()
                data[column] = data[column].fillna(fill_value)
                action = f"Filled with median ({fill_value:.2f})"
            elif fill_method == 'rolling_median':
                # Use rolling median strategy
                data[column] = fill_missing_with_rolling_median(data, date_column, column, rolling_window_days)
                action = f"Filled with rolling median (window={rolling_window_days} days)"
            else:
                raise ValueError("Invalid fill method. Choose 'mean', 'median', or 'rolling_median'.")
        else:
            # Drop the column if missing percentage is above the threshold
            data.drop(columns=[column], inplace=True)
            action = f"Dropped (missing > {threshold}%)"

        results.append({
            "Column": column,
            "Missing Count": missing_count,
            "Missing Percentage": missing_percentage,
            "Action": action
        })

    results_df = pd.DataFrame(results)
    return data, results_df

In [7]:
# Drop unnecessary columns
columns_to_drop = ['symbol', 'token', 'hour', 'day']
df = df.drop(columns=columns_to_drop, errors='ignore')

In [8]:
# Convert 'date' to datetime and extract useful features
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df['hour'] = df['date'].dt.hour  # Extract hour of the day
    df['day_of_week'] = df['date'].dt.dayofweek  # Extract day of the week (0=Monday, 6=Sunday)
    df['date'] = df['date'].astype('int64') / 10**9  # Convert to timestamp

In [9]:
df, missing_report = handle_missing_values(df)
print("Missing values in the dataset")
display(missing_report)

Missing values in the dataset


,Column,Missing Count,Missing Percentage,Action
0,date,0,0.000000,No Missing Values
1,open,0,0.000000,No Missing Values
2,high,0,0.000000,No Missing Values
3,low,0,0.000000,No Missing Values
4,close,0,0.000000,No Missing Values
5,Volume USDT,0,0.000000,No Missing Values
6,tradecount,0,0.000000,No Missing Values
7,ema_5,0,0.000000,No Missing Values
8,ema_15,0,0.000000,No Missing Values
9,ema_30,0,0.000000,No Missing Values


In [10]:
# Ensure essential columns are present before feature engineering
required_columns = ['open', 'close', 'Volume USDT', 'tradecount', 'high', 'low']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise KeyError(f"Missing required columns for feature engineering: {missing_columns}")

In [11]:
# Create the 'price' column as the row-wise median of 'open', 'close', 'high', and 'low'
df['price'] = df[['open', 'close', 'high', 'low']].median(axis=1)

# Move price at second column position
price_column = df.pop('price')
df.insert(1, 'price', price_column)

In [12]:
# Calculate price_direction based on the previous row's price
df['price_direction'] = (df['price'].diff().fillna(0) >= 0).astype(int) # 1: goes  up or same, 0: goes down

In [13]:
# Define features and target
# Here we drop 'price_direction' from features
features = df.drop(['price_direction'], axis=1)
target = df['price_direction']

In [14]:
# Safeguard the 'price' and 'price_direction' columns
prices = features['price'].values if 'price' in features.columns else None
predictions = target.values if 'price_direction' in target.name else None

In [16]:
# Feature engineering
# Calculate price_change_ratio based on the 'price' column
features['price_change_ratio'] = features['price'].pct_change().fillna(0)

# Calculate trade_volume_ratio safely
features['tradecount'] = features['tradecount'].fillna(0)  # Replace NaN with 0
features['trade_volume_ratio'] = np.where(
    features['tradecount'] == 0,  # Avoid division by zero
    features['Volume USDT'].median() / 1,  # Use median volume as fallback
    features['Volume USDT'] / features['tradecount']
)
features['trade_volume_ratio'] = features['trade_volume_ratio'].fillna(features['trade_volume_ratio'].median())

# Calculate high_low_spread as the difference between high and low prices
features['high_low_spread'] = features['high'] - features['low']

In [17]:
# Drop opem, close, high and low columns
columns_to_drop = ['open', 'close', 'high', 'low']
df = df.drop(columns=columns_to_drop, errors='ignore')

In [18]:
# Drop redundant features based on correlation
print("Analyzing feature correlations...")

# Define a threshold for high correlation
correlation_threshold = 0.9

# Compute the correlation matrix
correlation_matrix = features.corr()

# Create a set to hold the features to drop
features_to_drop = set()

# Iterate through the correlation matrix to find features with high correlation
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            col_name = correlation_matrix.columns[i]  # Feature to drop
            features_to_drop.add(col_name)

# Drop the identified features
if features_to_drop:
    features = features.drop(columns=features_to_drop, errors='ignore')
    print(
        f"\nThe following features were dropped due to high correlation "
        f"(threshold: {correlation_threshold * 100:.0f}%):"
    )
    for feature in features_to_drop:
        print(f"- {feature}")
else:
    print("\nNo features were dropped due to high correlation.")

Analyzing feature correlations...

The following features were dropped due to high correlation (threshold: 90%):
- MACD_Hist
- close
- low
- open
- ema_200
- ema_100
- Z-Score
- ema_5
- ema_30
- KAMA
- WMA
- ema_60
- ema_15
- HMA
- high


In [21]:
# Feature selection using RandomForest with cross-validation (optimized)
print("\nPerforming feature selection using RandomForestClassifier...")

# Initialize RandomForest with optimized parameters
clf = RandomForestClassifier(
    n_estimators=50,  # Reduce the number of trees (default is 100)
    max_depth=10,     # Limit the depth of trees to prevent overfitting
    random_state=42,
    n_jobs=-1         # Use all available CPU cores for parallel processing
)

# Use a smaller subset of input_data for cross-validation (if the dataset is large)
subset_size = min(50000, len(features))  # Use up to 50,000 samples for speed
if len(features) > subset_size:
    subset_indices = np.random.choice(len(features), subset_size, replace=False)
    features_subset = features.iloc[subset_indices]
    target_subset = target.iloc[subset_indices]
else:
    features_subset = features
    target_subset = target

# Perform cross-validation on the subset
scores = cross_val_score(clf, features_subset, target_subset, cv=3, scoring='accuracy')  # Reduce cv to 3
print(f"\nCross-validation accuracy scores: {scores}")
print(f"\nMean accuracy: {scores.mean():.4f}")

# Fit the model on the full dataset
clf.fit(features, target)

# Select relevant features based on importance
selector = SelectFromModel(clf, prefit=True, threshold="0.01*mean")
selected_feature_names = features.columns[selector.get_support()]
features = features[selected_feature_names]

# Display selected features
if not selected_feature_names.empty:
    print("\nThe following features were selected based on feature importance:")
    for feature in selected_feature_names:
        print(f"- {feature}")
else:
    print("\nNo features were selected by RandomForest feature selection.")

print("\nFeature selection process completed.")


Performing feature selection using RandomForestClassifier...

Cross-validation accuracy scores: [0.99994 0.99994 0.99988]

Mean accuracy: 0.9999

The following features were selected based on feature importance:
- date
- price
- Volume USDT
- tradecount
- MACD
- MACD_Signal
- ATR
- CMO
- QStick
- price_change_ratio
- high_low_spread

Feature selection process completed.


In [22]:
# Convert features and target to NumPy arrays
X = features.values
y = target.values

In [23]:
# Ensure the features and target are aligned
features = pd.DataFrame(X, columns=features.columns)  # Features from preprocessing
target = pd.Series(y, name="price_direction")         # Target column

In [24]:
# Display the shape of the dataset
print(f"\nShape of X: {X.shape}")


Shape of X: (1997210, 11)


In [25]:
# Display the first few rows of X (features)
display(features)

,date,price,Volume USDT,tradecount,MACD,MACD_Signal,ATR,CMO,QStick,price_change_ratio,high_low_spread
0,1.577837e+09,7180.720,509146.0,140.0,0.728704,-0.152219,4.684925,4.193879,0.120,0.000000,3.69
1,1.577837e+09,7178.470,713540.0,148.0,0.736887,-0.182091,4.698380,0.859360,0.528,-0.000313,3.76
2,1.577837e+09,7179.440,497793.0,104.0,0.846578,-0.117923,4.609025,11.466626,0.493,0.000135,5.60
3,1.577837e+09,7177.175,698627.0,193.0,0.650488,-0.343494,4.398181,-7.962104,-0.425,-0.000315,6.16
4,1.577837e+09,7175.160,241980.0,124.0,0.987398,-0.092457,4.262656,-6.795307,-0.131,-0.000281,3.86
...,...,...,...,...,...,...,...,...,...,...,...
1997205,1.698019e+09,29966.285,482950.0,635.0,-4056.925846,-1909.922487,926.278065,-97.689989,13.189,-0.000401,5.00
1997206,1.698019e+09,29970.500,169682.0,450.0,-3657.565528,-1988.042791,996.882531,-97.688910,24.399,0.000141,5.83
1997207,1.698019e+09,29975.100,111271.0,303.0,-3095.229187,-1922.717147,1072.856572,-97.688328,32.045,0.000153,3.40
1997208,1.698019e+09,29980.890,169741.0,631.0,-2332.807178,-1640.974425,1154.492462,-97.687019,22.669,0.000193,8.21


In [26]:
# Display the description of the dataset
print(f"\nShape of y: {y.shape}")


Shape of y: (1997210,)


In [27]:
# Display a table with the first few rows of y (target)
display(target.head())

0    1
1    0
2    1
3    0
4    0
Name: price_direction, dtype: int64